In [19]:
from manim import *
import numpy as np


class MinkowskiWave3D(ThreeDScene):
    def construct(self):
        # カメラ設定
        self.set_camera_orientation(phi=60 * DEGREES, theta=-45 * DEGREES)
        
        # 波のパラメータ
        A = 1.5  # 振幅
        k = 1.0  # 波数
        omega = np.pi/2  # 角周波数
        phi = 0  # 初期位相
        
        # 時空の範囲
        x_range = [-2*np.pi, 2*np.pi]
        t_range = [0, 4*np.pi]
        
        # 3D座標軸
        axes = ThreeDAxes(
            x_range=[x_range[0], x_range[1], np.pi],
            y_range=[t_range[0], t_range[1], np.pi],
            z_range=[-2, 2, 1],
            x_length=8,
            y_length=6,
            z_length=4,
            axis_config={"include_tip": True}
        )
        
        # 軸ラベル
        x_label = MathTex("x").next_to(axes.x_axis, RIGHT)
        t_label = MathTex("t").next_to(axes.y_axis, UP)
        psi_label = MathTex(r"\Psi").next_to(axes.z_axis, UP)
        
        # 波動関数の表示（実部）
        def wave_func(x, t):
            return A * np.cos(k * x - omega * t + phi)
        
        # 3D曲面として波を生成
        resolution = 30
        surface = Surface(
            lambda u, v: axes.c2p(u, v, wave_func(u, v)),
            u_range=x_range,
            v_range=t_range,
            resolution=(resolution, resolution),
            checkerboard_colors=[BLUE_D, BLUE_E],
            fill_opacity=0.7,
            stroke_width=0.5
        )
        
        # t=0での断面（xを変化）
        t_fixed = 0
        x_curve = ParametricFunction(
            lambda x: axes.c2p(x, t_fixed, wave_func(x, t_fixed)),
            t_range=x_range,
            color=RED,
            stroke_width=4
        )
        
        # x=0での断面（tを変化）
        x_fixed = 0
        t_curve = ParametricFunction(
            lambda t: axes.c2p(x_fixed, t, wave_func(x_fixed, t)),
            t_range=t_range,
            color=GREEN,
            stroke_width=4
        )
        
        # 移動する点
        # x方向に移動する点（t=0固定）
        x_dot = Dot3D(
            axes.c2p(x_range[0], t_fixed, wave_func(x_range[0], t_fixed)),
            color=RED,
            radius=0.08
        )
        
        # t方向に移動する点（x=0固定）
        t_dot = Dot3D(
            axes.c2p(x_fixed, t_range[0], wave_func(x_fixed, t_range[0])),
            color=GREEN,
            radius=0.08
        )
        
        # 複素平面（投影先）の設定
        # x移動の投影先（画面左下）
        complex_plane_x = ComplexPlane(
            x_range=[-2, 2],
            y_range=[-2, 2],
            x_length=3,
            y_length=3,
            background_line_style={
                "stroke_color": GREY,
                "stroke_width": 1,
                "stroke_opacity": 0.5
            }
        ).to_corner(DL, buff=0.5)
        
        # t移動の投影先（画面右下）
        complex_plane_t = ComplexPlane(
            x_range=[-2, 2],
            y_range=[-2, 2],
            x_length=3,
            y_length=3,
            background_line_style={
                "stroke_color": GREY,
                "stroke_width": 1,
                "stroke_opacity": 0.5
            }
        ).to_corner(DR, buff=0.5)
        
        # 複素平面のラベル
        x_plane_label = Text("t=0 (x varies)", font_size=20).next_to(complex_plane_x, UP)
        t_plane_label = Text("x=0 (t varies)", font_size=20).next_to(complex_plane_t, UP)
        
        # 投影ベクトル（初期位置）
        x_vector = Arrow(
            complex_plane_x.get_center(),
            complex_plane_x.n2p(complex(A * np.cos(k * x_range[0] + phi), 
                                       A * np.sin(k * x_range[0] + phi))),
            color=RED,
            stroke_width=3
        )
        x_vector_tip = Dot(x_vector.get_end(), color=RED, radius=0.06)
        
        t_vector = Arrow(
            complex_plane_t.get_center(),
            complex_plane_t.n2p(complex(A * np.cos(phi), 
                                       A * np.sin(phi))),
            color=GREEN,
            stroke_width=3
        )
        t_vector_tip = Dot(t_vector.get_end(), color=GREEN, radius=0.06)
        
        # タイトル
        title = Text("Wave in Minkowski Spacetime", font_size=30).to_edge(UP)
        equation = MathTex(
            r"\Psi(x,t) = A e^{i(kx - \omega t + \phi)}",
            font_size=24
        ).next_to(title, DOWN)
        
        # シーンに追加
        self.add(axes, x_label, t_label, psi_label)
        self.add(surface)
        self.add(x_curve, t_curve)
        self.add(x_dot, t_dot)
        self.add(complex_plane_x, complex_plane_t)
        self.add(x_plane_label, t_plane_label)
        self.add(x_vector, x_vector_tip, t_vector, t_vector_tip)
        self.add_fixed_in_frame_mobjects(title, equation)
        self.add_fixed_in_frame_mobjects(complex_plane_x, complex_plane_t)
        self.add_fixed_in_frame_mobjects(x_plane_label, t_plane_label)
        self.add_fixed_in_frame_mobjects(x_vector, x_vector_tip, t_vector, t_vector_tip)
        
        # アニメーション
        # x方向の移動
        x_path = ParametricFunction(
            lambda x: axes.c2p(x, t_fixed, wave_func(x, t_fixed)),
            t_range=x_range,
            color=RED
        )
        
        # t方向の移動
        t_path = ParametricFunction(
            lambda t: axes.c2p(x_fixed, t, wave_func(x_fixed, t)),
            t_range=t_range,
            color=GREEN
        )
        
        # 更新関数
        def update_x_vector(mob, alpha):
            x = interpolate(x_range[0], x_range[1], alpha)
            phase = k * x + phi
            new_end = complex_plane_x.n2p(complex(A * np.cos(phase), 
                                                  A * np.sin(phase)))
            mob.put_start_and_end_on(complex_plane_x.get_center(), new_end)
            
        def update_x_tip(mob, alpha):
            x = interpolate(x_range[0], x_range[1], alpha)
            phase = k * x + phi
            new_pos = complex_plane_x.n2p(complex(A * np.cos(phase), 
                                                 A * np.sin(phase)))
            mob.move_to(new_pos)
            
        def update_t_vector(mob, alpha):
            t = interpolate(t_range[0], t_range[1], alpha)
            phase = -omega * t + phi
            new_end = complex_plane_t.n2p(complex(A * np.cos(phase), 
                                                  A * np.sin(phase)))
            mob.put_start_and_end_on(complex_plane_t.get_center(), new_end)
            
        def update_t_tip(mob, alpha):
            t = interpolate(t_range[0], t_range[1], alpha)
            phase = -omega * t + phi
            new_pos = complex_plane_t.n2p(complex(A * np.cos(phase), 
                                                 A * np.sin(phase)))
            mob.move_to(new_pos)
        
        # カメラを少し動かす
        self.begin_ambient_camera_rotation(rate=0.1)
        
        # 同時アニメーション
        self.play(
            MoveAlongPath(x_dot, x_path),
            MoveAlongPath(t_dot, t_path),
            UpdateFromAlphaFunc(x_vector, update_x_vector),
            UpdateFromAlphaFunc(x_vector_tip, update_x_tip),
            UpdateFromAlphaFunc(t_vector, update_t_vector),
            UpdateFromAlphaFunc(t_vector_tip, update_t_tip),
            run_time=8,
            rate_func=linear
        )
        
        self.wait(2)

In [20]:
%manim -qh MinkowskiWave3D

Manim Community v0.19.0

[06/11/25 08:27:26] INFO     Animation 0 : Partial movie file written in                   ]8;id=547090;file:///workspaces/mictlan/.venv/lib/python3.12/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=813368;file:///workspaces/mictlan/.venv/lib/python3.12/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/workspaces/mictlan/research/syuron/docs/media/videos/docs/1                         
                             080p60/partial_movie_files/MinkowskiWave3D/3846668646_2997339                         
                             53_1261375191.mp4'                                                                    

[06/11/25 08:29:22] INFO     Animation 1 : Partial movie file written in                   ]8;id=846371;file:///workspaces/mictlan/.venv/lib/python3.12/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=729933;file:///workspaces/mictlan/.venv/lib/python3.12/site-packages/manim/scene/scene_file_writer.py#588\588]8;;\
                             '/workspaces/mictlan/research/syuron/docs/media/videos/docs/1                         
                             080p60/partial_movie_files/MinkowskiWave3D/253757304_24428107                         
                             29_3684541631.mp4'                                                                    

                    INFO     Combining to Movie file.                                      ]8;id=724156;file:///workspaces/mictlan/.venv/lib/python3.12/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=348495;file:///workspaces/mictlan/.venv/lib/python3.12/site-packages/manim/scene/scene_file_writer.py#739\739]8;;\

                    INFO                                                                   ]8;id=396734;file:///workspaces/mictlan/.venv/lib/python3.12/site-packages/manim/scene/scene_file_writer.py\scene_file_writer.py]8;;\:]8;id=973987;file:///workspaces/mictlan/.venv/lib/python3.12/site-packages/manim/scene/scene_file_writer.py#886\886]8;;\
                             File ready at                                                                         
                             '/workspaces/mictlan/research/syuron/docs/media/videos/docs/1                         
                             080p60/MinkowskiWave3D.mp4'                                                           
                                                                                                                   

                    INFO     Rendered MinkowskiWave3D                                                  ]8;id=114549;file:///workspaces/mictlan/.venv/lib/python3.12/site-packages/manim/scene/scene.py\scene.py]8;;\:]8;id=61222;file:///workspaces/mictlan/.venv/lib/python3.12/site-packages/manim/scene/scene.py#255\255]8;;\
                             Played 2 animations                                                                   